In [2]:
#!pip install ultralytics
import cv2
from ultralytics import YOLO
import os


In [11]:
currDirectory = os.getcwd()

#### Tracking sur les données de Moodle

In [20]:
frames_folder = "data/frames/"           # Folder which contains all the images
frame_files = os.listdir(frames_folder)   # List of image filenames
#frame_files.sort()  # Sort the list of files
frame_files =  sorted(frame_files, key=lambda x: int(x[5:-4]))

# Delete the file if it's already exist
if os.path.exists(currDirectory + '/results.txt'):
    os.remove(currDirectory + '/results.txt')

def detect_track_data():
    
    # Load the YOLOv8 model detection
    model = YOLO('yolov8n.pt')

    for frame_id, frame_file in enumerate(frame_files) :
        frame_path = os.path.join(frames_folder, frame_file)   # image path
        frame = cv2.imread(frame_path, cv2.IMREAD_COLOR)       # Charger et lire l'image
    
        if frame is not None:
            
            frame_id += 1
            
            # Run YOLOv8 tracking on the frame, persisting tracks between frames, selecting tracker and objet of interest = 41 = cup
            # Set show = True to see tracking results directly
            results = model.track(frame, imgsz=640, persist=True, tracker="bytetrack.yaml", classes=41, verbose=False, show=False, conf=0.25, iou=0.7)
            
            # Get the boxes and track IDs because in zip some id isn't present
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
            
            
            for box, track_id in zip(boxes, track_ids):
                x_min = int(box[0])
                y_min = int(box[1])
                x_max = int(box[2])
                y_max = int(box[3])
                width = x_max - x_min
                height = y_max - y_min
                # Save the results in results.txt
                with open(currDirectory + '/results.txt', 'a+') as file:
                    file.write(f"{frame_id} {track_id} {x_min} {y_min} {width} {height}\n")
            
        else:
            print(f"frame not found or impossible to read")

detect_track_data()


#### Tracking sur les jeux de données MOT17

In [14]:
# Open MOT17 video train image
mot17_folder = "MOT17/train"
directories = [d for d in os.listdir(mot17_folder)]


for dir in directories:
    
    
    
    images_path = f"{mot17_folder}/{dir}/img1"
    output_file = f"/TrackEval/data/trackers/mot_challenge/MOT17-train/ByteTrack/data/{dir}.txt"

    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

    # Sort the files by their digits, take off the word "frame" and the extension
    image_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    
    # Load the YOLOv8 model detection
    model = YOLO('yolov8n.pt')
    
    # Dictionnaire pour stocker les informations de détection
    detections_dict = {}  
    
    # Delate the output file if already exist
    if os.path.exists(currDirectory + output_file):
        os.remove(currDirectory + output_file)
    
    for image_id, image in enumerate(image_files):

        image = cv2.imread(os.path.join(images_path, image_files[image_id]))
        image_id += 1

        # Run YOLOv8 tracking on the frame, persisting tracks between frames, and select class pedestrian
        results = model.track(image, imgsz=640, data='coco128.yaml', persist=True, tracker="bytetrack.yaml", classes=[0], verbose=False, show=False, conf=0.25, iou=0.7)

        # Get the boxes and track IDs
        if results[0].boxes.id is not None and hasattr(results[0].boxes, 'id'):
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Ajouter les détections au dictionnaire detections_dict
            for box, track_id in zip(boxes, track_ids):
                x_min = box[0]
                y_min = box[1]
                x_max = box[2]
                y_max = box[3]
                width = x_max - x_min
                height = y_max - y_min

                if track_id not in detections_dict:
                    detections_dict[track_id] = []
                detections_dict[track_id].append((image_id, track_id, x_min, y_min, width, height))
                
                with open(currDirectory + output_file, 'a+') as file:
                    file.write(f"{image_id},{track_id},{x_min},{y_min},{width},{height},{1},{-1},{-1},{-1} \n")

        #else:
             #print("Ids are not available.")
        

     

    



In [15]:
import subprocess

# Exécuter une commande shell
subprocess.run(["python3", "TrackEval/scripts/run_mot_challenge.py", "--USE_PARALLEL", "False", "--METRICS", "HOTA", "--TRACKERS_TO_EVAL", "ByteTrack"])


Error importing BURST due to missing underlying dependency: No module named 'pycocotools'

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/ariste/Documents/Maitrise-Genie-Info/Vision-par-ordinateur/Chapitre4/TP3/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True             

CompletedProcess(args=['python3', 'TrackEval/scripts/run_mot_challenge.py', '--USE_PARALLEL', 'False', '--METRICS', 'HOTA', '--TRACKERS_TO_EVAL', 'ByteTrack'], returncode=0)